# tsfMRI Kernel Preprocessing

In [ ]:
# Set working directory
setwd("./WorkDir")

# Create holding directory for preprocessed datasets
if (!dir.exists("./CSI/Preprocessed")) {dir.create("./CSI/Preprocessed")}

# Load necessary libraries
suppressPackageStartupMessages(library(tidyverse))

### MRI QC Raw: _mriqcrp_

In [ ]:
# Import partial mriqcrp data
mriqcrp10301 <- read_delim("./ABCD_Data/mriqcrp10301.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

mriqcrp20301 <- read_delim("./ABCD_Data/mriqcrp20301.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

mriqcrp302 <- read_delim("./ABCD_Data/mriqcrp302.txt", 
                         delim = "\t", 
                         escape_double = FALSE, 
                         col_types = "c", 
                         trim_ws = TRUE, 
                         na = c("", "NA"))[-1,]

In [ ]:
# Load partial mriqcrp data and join

mriqcrp10301 <- mriqcrp10301 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("iqc")))

mriqcrp20301 <- mriqcrp20301 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("iqc")))

mriqcrp302 <- mriqcrp302 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("iqc")))

mriqcrp <- mriqcrp10301 %>%
    full_join(mriqcrp20301, by=c("subjectkey"="subjectkey")) %>%
    full_join(mriqcrp302, by=c("subjectkey"="subjectkey"))

In [ ]:
# Create a tsfMRI_mriqcrp subset
tsfMRI_mriqcrp <- mriqcrp %>%
    select(c(subjectkey, starts_with("iqc_mid_"), starts_with("iqc_sst_"), starts_with("iqc_nback_"))) %>%
    select(-c(ends_with("sub_02"), ends_with("seriestime"), ends_with("studydate"), ends_with("seuid"), ends_with("suid"), ends_with("_date"), ends_with("_time"))) %>%
    select(-matches("iqc_.{0,5}_\\d_.{0,}")) %>%
    mutate_at(., vars(contains("iqc_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(contains("iqc_")), mean, na.rm=T)

dim(tsfMRI_mriqcrp)
head(tsfMRI_mriqcrp)

In [ ]:
# Save tsfMRI_mriqcrp, remove intermediate files
saveRDS(tsfMRI_mriqcrp, file="./CSI/Preprocessed/tsfMRI_mriqcrp.rds")
rm(mriqcrp, mriqcrp10301, mriqcrp20301, mriqcrp302)

### Automated Post-Processing QC Metrics: _postqc_

In [ ]:
# Import partial postqc data
postqc01 <- read_delim("./ABCD_Data/abcd_auto_postqc01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_postqc subset
tsfMRI_postqc <- postqc01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("apqc_fmri_"))) %>%
    mutate_at(., vars(starts_with("apqc_fmri_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("apqc_fmri_")), mean, na.rm=T)

dim(tsfMRI_postqc)
head(tsfMRI_postqc)

In [ ]:
# Save tsfMRI_postqc, remove intermediate files
saveRDS(tsfMRI_postqc, file="./CSI/Preprocessed/tsfMRI_postqc.rds")
rm(postqc01)

### Manual fMRI Post-Processing QC: _fmriqc_

In [ ]:
# Import partial fmriqc data
abcd_fmriqc01 <- read_delim("./ABCD_Data/abcd_fmriqc01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a fmriqc subset
fmriqc <- abcd_fmriqc01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("fmri_postqc_"))) %>%
    mutate_at(., vars(starts_with("fmri_postqc_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("fmri_postqc_")), mean, na.rm=T)

dim(fmriqc)
head(fmriqc)

In [ ]:
# Save fmriqc, remove intermediate files
saveRDS(fmriqc, file="./CSI/Preprocessed/tsfMRI_fmriqc.rds")
rm(abcd_fmriqc01)

### ABCD Task fMRI MID Average Beta Weights Destrieux Parcellations: _midabwdp_

In [ ]:
# Import partial midabwdp data
abcd_midabwdp01 <- read_delim("./ABCD_Data/abcd_midabwdp01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_midabwdp202 <- read_delim("./ABCD_Data/abcd_midabwdp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midabwdp <- full_join(abcd_midabwdp01, abcd_midabwdp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midabwdp subset
tsfMRI_midabwdp <- midabwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("midab"))) %>%
    mutate_at(., vars(starts_with("midab")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("midab")), mean, na.rm=T)

dim(tsfMRI_midabwdp)
head(tsfMRI_midabwdp)

In [ ]:
# Save tsfMRI_midabwdp, remove intermediate files
saveRDS(tsfMRI_midabwdp, file="./CSI/Preprocessed/tsfMRI_midabwdp.rds")
rm(abcd_midabwdp01, abcd_midabwdp202, midabwdp)

### ABCD Task fMRI MID Average Beta Weights: _midaparc_

In [ ]:
# Import partial midaparc data
midaparc03 <- read_delim("./ABCD_Data/midaparc03.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midaparcp203 <- read_delim("./ABCD_Data/midaparcp203.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midaparcp <- full_join(midaparc03, midaparcp203, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midaparcp subset
tsfMRI_midaparcp <- midaparcp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri_"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri_")), mean, na.rm=T)

dim(tsfMRI_midaparcp)
head(tsfMRI_midaparcp)

In [ ]:
# Save tsfMRI_midaparcp, remove intermediate files
saveRDS(tsfMRI_midaparcp, file="./CSI/Preprocessed/tsfMRI_midaparcp.rds")
rm(midaparcp, midaparc03, midaparcp203)

### ABCD Task fMRI MID Average SEM Destrieux Parcellations: _midasemdp_

In [ ]:
# Import partial midasemdp data
abcd_midasemdp101 <- read_delim("./ABCD_Data/abcd_midasemdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_midasemdp202 <- read_delim("./ABCD_Data/abcd_midasemdp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midasemdp <- full_join(abcd_midasemdp101, abcd_midasemdp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midasemdp subset
tsfMRI_midasemdp <- midasemdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("midasemdp"))) %>%
    mutate_at(., vars(starts_with("midasemdp")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("midasemdp")), mean, na.rm=T)

dim(tsfMRI_midasemdp)
head(tsfMRI_midasemdp)

In [ ]:
# Save tsfMRI_midasemdp, remove intermediate files
saveRDS(tsfMRI_midasemdp, file="./CSI/Preprocessed/tsfMRI_midasemdp.rds")
rm(abcd_midasemdp101, abcd_midasemdp202, midasemdp)

### ABCD Task fMRI MID Average Standard Error of the Mean: _midasemp_

In [ ]:
# Import partial midasemp data
abcd_midasemp102 <- read_delim("./ABCD_Data/abcd_midasemp102.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_midasemp202 <- read_delim("./ABCD_Data/abcd_midasemp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midasemp <- full_join(abcd_midasemp102, abcd_midasemp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midasemp subset
tsfMRI_midasemp <- midasemp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri_"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri_")), mean, na.rm=T)

dim(tsfMRI_midasemp)
head(tsfMRI_midasemp)

In [ ]:
# Save tsfMRI_midasemp, remove intermediate files
saveRDS(tsfMRI_midasemp, file="./CSI/Preprocessed/tsfMRI_midasemp.rds")
rm(abcd_midasemp102, abcd_midasemp202, midasemp)

### ABCD Task fMRI MID Run 1 Beta Weights Destrieux Parcellations: _midr1bwdp_

In [ ]:
# Import partial midr1bwdp data
abcd_midr1bwdp101 <- read_delim("./ABCD_Data/abcd_midr1bwdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_midr1bwdp202 <- read_delim("./ABCD_Data/abcd_midr1bwdp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midr1bwdp <- full_join(abcd_midr1bwdp101, abcd_midr1bwdp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midr1bwdp subset
tsfMRI_midr1bwdp <- midr1bwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmrimr1_"))) %>%
    mutate_at(., vars(starts_with("tfmrimr1_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmrimr1_")), mean, na.rm=T)

dim(tsfMRI_midr1bwdp)
head(tsfMRI_midr1bwdp)

In [ ]:
# Save tsfMRI_midr1bwdp, remove intermediate files
saveRDS(tsfMRI_midr1bwdp, file="./CSI/Preprocessed/tsfMRI_midr1bwdp.rds")
rm(abcd_midr1bwdp101, abcd_midr1bwdp202, midr1bwdp)

### ABCD Task fMRI MID Run 1 Beta Weights: _midr1bwp_

In [ ]:
# Import partial midr1bwp data
abcd_midr1bwp102 <- read_delim("./ABCD_Data/abcd_midr1bwp102.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_midr1bwp202 <- read_delim("./ABCD_Data/abcd_midr1bwp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midr1bwp <- full_join(abcd_midr1bwp102, abcd_midr1bwp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midr1bwp subset
tsfMRI_midr1bwp <- midr1bwp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri_"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri_")), mean, na.rm=T)

dim(tsfMRI_midr1bwp)
head(tsfMRI_midr1bwp)

In [ ]:
# Save tsfMRI_midr1bwp, remove intermediate files
saveRDS(tsfMRI_midr1bwp, file="./CSI/Preprocessed/tsfMRI_midr1bwp.rds")
rm(abcd_midr1bwp102, abcd_midr1bwp202, midr1bwp)

### ABCD Task fMRI MID Run 1 SEM Destrieux Parcellations: _tmidr1semdp_

In [ ]:
# Import partial tmidr1semdp data
abcd_tmidr1semdp101 <- read_delim("./ABCD_Data/abcd_tmidr1semdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tmidr1semdp202 <- read_delim("./ABCD_Data/abcd_tmidr1semdp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tmidr1semdp <- full_join(abcd_tmidr1semdp101, abcd_tmidr1semdp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tmidr1semdp subset
tsfMRI_tmidr1semdp <- tmidr1semdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tmidr1semdp_"))) %>%
    mutate_at(., vars(starts_with("tmidr1semdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tmidr1semdp_")), mean, na.rm=T)

dim(tsfMRI_tmidr1semdp)
head(tsfMRI_tmidr1semdp)

In [ ]:
# Save tsfMRI_tmidr1semdp, remove intermediate files
saveRDS(tsfMRI_tmidr1semdp, file="./CSI/Preprocessed/tsfMRI_tmidr1semdp.rds")
rm(abcd_tmidr1semdp101, abcd_tmidr1semdp202, tmidr1semdp)

### ABCD Task fMRI MID Run 1 Standard Error of the Mean: _midsemp_

In [ ]:
# Import partial midsemp data
abcd_midsemp102 <- read_delim("./ABCD_Data/abcd_midsemp102.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_midsemp202 <- read_delim("./ABCD_Data/abcd_midsemp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midsemp <- full_join(abcd_midsemp102, abcd_midsemp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midsemp subset
tsfMRI_midsemp <- midsemp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri_"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri_")), mean, na.rm=T)

dim(tsfMRI_midsemp)
head(tsfMRI_midsemp)

In [ ]:
# Save tsfMRI_midsemp, remove intermediate files
saveRDS(tsfMRI_midsemp, file="./CSI/Preprocessed/tsfMRI_midsemp.rds")
rm(abcd_midsemp102, abcd_midsemp202, midsemp)

### ABCD Task fMRI MID Run 2 Beta Weights Destrieux Parcellations: _tr2bwdp_

In [ ]:
# Import partial tr2bwdp data
abcd_tr2bwdp01 <- read_delim("./ABCD_Data/abcd_tr2bwdp01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tr2bwdp202 <- read_delim("./ABCD_Data/abcd_tr2bwdp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tr2bwdp <- full_join(abcd_tr2bwdp01, abcd_tr2bwdp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tr2bwdp subset
tsfMRI_tr2bwdp <- tr2bwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tr2bwdp_"))) %>%
    mutate_at(., vars(starts_with("tr2bwdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tr2bwdp_")), mean, na.rm=T)

dim(tsfMRI_tr2bwdp)
head(tsfMRI_tr2bwdp)

In [ ]:
# Save tsfMRI_tr2bwdp, remove intermediate files
saveRDS(tsfMRI_tr2bwdp, file="./CSI/Preprocessed/tsfMRI_tr2bwdp.rds")
rm(abcd_tr2bwdp01, abcd_tr2bwdp202, tr2bwdp)

### ABCD Task fMRI MID Run 2 Beta Weights: _midr2bwp_

In [ ]:
# Import partial midr2bwp data
midr2bwp102 <- read_delim("./ABCD_Data/midr2bwp102.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midr2bwp202 <- read_delim("./ABCD_Data/midr2bwp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midr2bwp <- full_join(midr2bwp102, midr2bwp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midr2bwp subset
tsfMRI_midr2bwp <- midr2bwp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri_"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri_")), mean, na.rm=T)

dim(tsfMRI_midr2bwp)
head(tsfMRI_midr2bwp)

In [ ]:
# Save tsfMRI_midr2bwp, remove intermediate files
saveRDS(tsfMRI_midr2bwp, file="./CSI/Preprocessed/tsfMRI_midr2bwp.rds")
rm(midr2bwp102, midr2bwp202, midr2bwp)

### ABCD Task fMRI MID Run 2 SEM Destrieux Parcellations: _tr2semdp_

In [ ]:
# Import partial tr2semdp data
abcd_tr2semdp101 <- read_delim("./ABCD_Data/abcd_tr2semdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tr2semdp202 <- read_delim("./ABCD_Data/abcd_tr2semdp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tr2semdp <- full_join(abcd_tr2semdp101, abcd_tr2semdp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tr2semdp subset
tsfMRI_tr2semdp <- tr2semdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tr2semdp_"))) %>%
    mutate_at(., vars(starts_with("tr2semdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tr2semdp_")), mean, na.rm=T)

dim(tsfMRI_tr2semdp)
head(tsfMRI_tr2semdp)

In [ ]:
# Save tsfMRI_tr2semdp, remove intermediate files
saveRDS(tsfMRI_tr2semdp, file="./CSI/Preprocessed/tsfMRI_tr2semdp.rds")
rm(abcd_tr2semdp101, abcd_tr2semdp202, tr2semdp)

### ABCD Task fMRI MID Run 2 Standard Error of the Mean: _midr2semp_

In [ ]:
# Import partial midr2semp data
abcd_midr2semp102 <- read_delim("./ABCD_Data/abcd_midr2semp102.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_midr2semp202 <- read_delim("./ABCD_Data/abcd_midr2semp202.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

midr2semp <- full_join(abcd_midr2semp102, abcd_midr2semp202, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_midr2semp subset
tsfMRI_midr2semp <- midr2semp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_midr2semp)
head(tsfMRI_midr2semp)

In [ ]:
# Save tsfMRI_midr2semp, remove intermediate files
saveRDS(tsfMRI_midr2semp, file="./CSI/Preprocessed/tsfMRI_midr2semp.rds")
rm(abcd_midr2semp102, abcd_midr2semp202, midr2semp)

### ABCD Task fMRI SST Average Beta Weights: _mrisst02_

In [ ]:
# Import partial mrisst02 data
mrisst02 <- read_delim("./ABCD_Data/mrisst02.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a mrisst02 subset
tsfMRI_mrisst02 <- mrisst02 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_mrisst02)
head(tsfMRI_mrisst02)

In [ ]:
# Save tsfMRI_mrisst02, remove intermediate files
saveRDS(tsfMRI_mrisst02, file="./CSI/Preprocessed/tsfMRI_mrisst02.rds")
rm(mrisst02)

### ABCD Task fMRI SST Average Beta Weights Destrieux Parcellations: _tfsstabwdp_

In [ ]:
# Import partial tfsstabwdp data
abcd_tfsstabwdp101 <- read_delim("./ABCD_Data/abcd_tfsstabwdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfsstabwdp201 <- read_delim("./ABCD_Data/abcd_tfsstabwdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfsstabwdp <- full_join(abcd_tfsstabwdp101, abcd_tfsstabwdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfsstabwdp subset
tsfMRI_tfsstabwdp <- tfsstabwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfsstabwdp_"))) %>%
    mutate_at(., vars(starts_with("tfsstabwdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfsstabwdp_")), mean, na.rm=T)

dim(tsfMRI_tfsstabwdp)
head(tsfMRI_tfsstabwdp)

In [ ]:
# Save tsfMRI_tfsstabwdp, remove intermediate files
saveRDS(tsfMRI_tfsstabwdp, file="./CSI/Preprocessed/tsfMRI_tfsstabwdp.rds")
rm(abcd_tfsstabwdp101, abcd_tfsstabwdp201, tfsstabwdp)

### ABCD Task fMRI SST Average SEM Destrieux Parcellations: _tfsstasemdp_

In [ ]:
# Import partial tfsstabwdp data
abcd_tfsstasemdp101 <- read_delim("./ABCD_Data/abcd_tfsstasemdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfsstasemdp201 <- read_delim("./ABCD_Data/abcd_tfsstasemdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfsstasemdp <- full_join(abcd_tfsstasemdp101, abcd_tfsstasemdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfsstasemdp subset
tsfMRI_tfsstasemdp <- tfsstasemdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfsstasemdp_"))) %>%
    mutate_at(., vars(starts_with("tfsstasemdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfsstasemdp_")), mean, na.rm=T)

dim(tsfMRI_tfsstasemdp)
head(tsfMRI_tfsstasemdp)

In [ ]:
# Save tsfMRI_tfsstasemdp, remove intermediate files
saveRDS(tsfMRI_tfsstasemdp, file="./CSI/Preprocessed/tsfMRI_tfsstasemdp.rds")
rm(abcd_tfsstasemdp101, abcd_tfsstasemdp201, tfsstasemdp)

### ABCD Task fMRI SST Average Standard Error of the Mean: _mrisstsem01_

In [ ]:
# Import partial mrisstsem01 data
mrisstsem01 <- read_delim("./ABCD_Data/mrisstsem01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_mrisstsem01 subset
tsfMRI_mrisstsem01 <- mrisstsem01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_mrisstsem01)
head(tsfMRI_mrisstsem01)

In [ ]:
# Save tsfMRI_mrisstsem01, remove intermediate files
saveRDS(tsfMRI_mrisstsem01, file="./CSI/Preprocessed/tsfMRI_mrisstsem01.rds")
rm(mrisstsem01)

### ABCD Task fMRI SST Run 1 Beta Weights: _mrisstr1bw01_

In [ ]:
# Import partial mrisstr1bw01 data
mrisstr1bw01 <- read_delim("./ABCD_Data/mrisstr1bw01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_mrisstr1bw01 subset
tsfMRI_mrisstr1bw01 <- mrisstr1bw01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_mrisstr1bw01)
head(tsfMRI_mrisstr1bw01)

In [ ]:
# Save tsfMRI_mrisstr1bw01, remove intermediate files
saveRDS(tsfMRI_mrisstr1bw01, file="./CSI/Preprocessed/tsfMRI_mrisstr1bw01.rds")
rm(mrisstr1bw01)

### ABCD Task fMRI SST Run 1 Beta Weights Destrieux Parcellations: _tfsstr1bwdp_

In [ ]:
# Import partial tfsstr1bwdp data
abcd_tfsstr1bwdp101 <- read_delim("./ABCD_Data/abcd_tfsstr1bwdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfsstr1bwdp201 <- read_delim("./ABCD_Data/abcd_tfsstr1bwdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfsstr1bwdp <- full_join(abcd_tfsstr1bwdp101, abcd_tfsstr1bwdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfsstr1bwdp subset
tsfMRI_tfsstr1bwdp <- tfsstr1bwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfsstr1bwdp_"))) %>%
    mutate_at(., vars(starts_with("tfsstr1bwdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfsstr1bwdp_")), mean, na.rm=T)

dim(tsfMRI_tfsstr1bwdp)
head(tsfMRI_tfsstr1bwdp)

In [ ]:
# Save tsfMRI_tfsstr1bwdp, remove intermediate files
saveRDS(tsfMRI_tfsstr1bwdp, file="./CSI/Preprocessed/tsfMRI_tfsstr1bwdp.rds")
rm(abcd_tfsstr1bwdp101, abcd_tfsstr1bwdp201, tfsstr1bwdp)

### ABCD Task fMRI SST Run 1 SEM Destrieux Parcellations: _tfsstr1semdp_

In [ ]:
# Import partial tfsstr1semdp data
abcd_tfsstr1semdp101 <- read_delim("./ABCD_Data/abcd_tfsstr1semdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfsstr1semdp201 <- read_delim("./ABCD_Data/abcd_tfsstr1semdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfsstr1semdp <- full_join(abcd_tfsstr1semdp101, abcd_tfsstr1semdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfsstr1semdp subset
tsfMRI_tfsstr1semdp <- tfsstr1semdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfsstr1semdp_"))) %>%
    mutate_at(., vars(starts_with("tfsstr1semdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfsstr1semdp_")), mean, na.rm=T)

dim(tsfMRI_tfsstr1semdp)
head(tsfMRI_tfsstr1semdp)

In [ ]:
# Save tsfMRI_tfsstr1semdp, remove intermediate files
saveRDS(tsfMRI_tfsstr1semdp, file="./CSI/Preprocessed/tsfMRI_tfsstr1semdp.rds")
rm(abcd_tfsstr1semdp101, abcd_tfsstr1semdp201, tfsstr1semdp)

### ABCD Task fMRI SST Run 1 Standard Error of the Mean: _mrisstr1sem01_

In [ ]:
# Import partial mrisstr1bw01 data
mrisstr1sem01 <- read_delim("./ABCD_Data/mrisstr1sem01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_mrisstr1sem01 subset
tsfMRI_mrisstr1sem01 <- mrisstr1sem01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_mrisstr1sem01)
head(tsfMRI_mrisstr1sem01)

In [ ]:
# Save tsfMRI_mrisstr1sem01, remove intermediate files
saveRDS(tsfMRI_mrisstr1sem01, file="./CSI/Preprocessed/tsfMRI_mrisstr1sem01.rds")
rm(mrisstr1sem01)

### ABCD Task fMRI SST Run 2 Beta Weights: _mrisstr2bw01_

In [ ]:
# Import partial mrisstr2bw01 data
mrisstr2bw01 <- read_delim("./ABCD_Data/mrisstr2bw01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_mrisstr2bw01 subset
tsfMRI_mrisstr2bw01 <- mrisstr2bw01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_mrisstr2bw01)
head(tsfMRI_mrisstr2bw01)

In [ ]:
# Save tsfMRI_mrisstr2bw01, remove intermediate files
saveRDS(tsfMRI_mrisstr2bw01, file="./CSI/Preprocessed/tsfMRI_mrisstr2bw01.rds")
rm(mrisstr2bw01)

### ABCD Task fMRI SST Run 2 Beta Weights Destrieux Parcellations: _tfsstr2bwdp_

In [ ]:
# Import partial tfsstr1semdp data
abcd_tfsstr2bwdp101 <- read_delim("./ABCD_Data/abcd_tfsstr2bwdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfsstr2bwdp201 <- read_delim("./ABCD_Data/abcd_tfsstr2bwdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfsstr2bwdp <- full_join(abcd_tfsstr2bwdp101, abcd_tfsstr2bwdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfsstr2bwdp subset
tsfMRI_tfsstr2bwdp <- tfsstr2bwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfsstr2bwdp_"))) %>%
    mutate_at(., vars(starts_with("tfsstr2bwdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfsstr2bwdp_")), mean, na.rm=T)

dim(tsfMRI_tfsstr2bwdp)
head(tsfMRI_tfsstr2bwdp)

In [ ]:
# Save tsfMRI_tfsstr2bwdp, remove intermediate files
saveRDS(tsfMRI_tfsstr2bwdp, file="./CSI/Preprocessed/tsfMRI_tfsstr2bwdp.rds")
rm(abcd_tfsstr2bwdp101, abcd_tfsstr2bwdp201, tfsstr2bwdp)

### ABCD Task fMRI SST Run 2 SEM Destrieux Parcellations: _tfsstr2semdp_

In [ ]:
# Import partial tfsstr2semdp data
abcd_tfsstr2semdp101 <- read_delim("./ABCD_Data/abcd_tfsstr2semdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfsstr2semdp201 <- read_delim("./ABCD_Data/abcd_tfsstr2semdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfsstr2semdp <- full_join(abcd_tfsstr2semdp101, abcd_tfsstr2semdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfsstr2semdp subset
tsfMRI_tfsstr2semdp <- tfsstr2semdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfsstr2semdp_"))) %>%
    mutate_at(., vars(starts_with("tfsstr2semdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfsstr2semdp_")), mean, na.rm=T)

dim(tsfMRI_tfsstr2semdp)
head(tsfMRI_tfsstr2semdp)

In [ ]:
# Save tsfMRI_tfsstr2semdp, remove intermediate files
saveRDS(tsfMRI_tfsstr2semdp, file="./CSI/Preprocessed/tsfMRI_tfsstr2semdp.rds")
rm(abcd_tfsstr2semdp101, abcd_tfsstr2semdp201, tfsstr2semdp)

### ABCD Task fMRI SST Run 2 Standard Error of the Mean: _mrisstr2bwsem01_

In [ ]:
# Import partial mrisstr2bwsem01 data
mrisstr2bwsem01 <- read_delim("./ABCD_Data/mrisstr2bwsem01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_mrisstr2bwsem01 subset
tsfMRI_mrisstr2bwsem01 <- mrisstr2bwsem01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_mrisstr2bwsem01)
head(tsfMRI_mrisstr2bwsem01)

In [ ]:
# Save tsfMRI_mrisstr2bwsem01, remove intermediate files
saveRDS(tsfMRI_mrisstr2bwsem01, file="./CSI/Preprocessed/tsfMRI_mrisstr2bwsem01.rds")
rm(mrisstr2bwsem01)

### ABCD Task fMRI nBack Average Beta Weights: _nback_bwroi02_

In [ ]:
# Import partial nback_bwroi02 data
nback_bwroi02 <- read_delim("./ABCD_Data/nback_bwroi02.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_nback_bwroi02 subset
tsfMRI_nback_bwroi02 <- nback_bwroi02 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_nback_bwroi02)
head(tsfMRI_nback_bwroi02)

In [ ]:
# Save tsfMRI_nback_bwroi02, remove intermediate files
saveRDS(tsfMRI_nback_bwroi02, file="./CSI/Preprocessed/tsfMRI_nback_bwroi02.rds")
rm(nback_bwroi02)

### ABCD Task fMRI nBack Average Beta Weights Destrieux Parcellations: _tfabwdp_

In [ ]:
# Import partial tfabwdp data
abcd_tfabwdp101 <- read_delim("./ABCD_Data/abcd_tfabwdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfabwdp201 <- read_delim("./ABCD_Data/abcd_tfabwdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfabwdp <- full_join(abcd_tfabwdp101, abcd_tfabwdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfabwdp subset
tsfMRI_tfabwdp <- tfabwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfabwdp_"))) %>%
    mutate_at(., vars(starts_with("tfabwdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfabwdp_")), mean, na.rm=T)

dim(tsfMRI_tfabwdp)
head(tsfMRI_tfabwdp)

In [ ]:
# Save tsfMRI_tfabwdp, remove intermediate files
saveRDS(tsfMRI_tfabwdp, file="./CSI/Preprocessed/tsfMRI_tfabwdp.rds")
rm(abcd_tfabwdp101, abcd_tfabwdp201, tfabwdp)

### ABCD Task fMRI nBack Average SEM Destrieux Parcellations: _tnbasemdp_

In [ ]:
# Import partial tnbasemdp data
abcd_tnbasemdp101 <- read_delim("./ABCD_Data/abcd_tnbasemdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tnbasemdp201 <- read_delim("./ABCD_Data/abcd_tnbasemdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tnbasemdp <- full_join(abcd_tnbasemdp101, abcd_tnbasemdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tnbasemdp subset
tsfMRI_tnbasemdp <- tnbasemdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tnbasemdp_"))) %>%
    mutate_at(., vars(starts_with("tnbasemdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tnbasemdp_")), mean, na.rm=T)

dim(tsfMRI_tnbasemdp)
head(tsfMRI_tnbasemdp)

In [ ]:
# Save tsfMRI_tnbasemdp, remove intermediate files
saveRDS(tsfMRI_tnbasemdp, file="./CSI/Preprocessed/tsfMRI_tnbasemdp.rds")
rm(abcd_tnbasemdp101, abcd_tnbasemdp201, tnbasemdp)

### ABCD Task fMRI nBack Average Standard Error of the Mean: _nbackallsem01_

In [ ]:
# Import partial nbackallsem01 data
nbackallsem01 <- read_delim("./ABCD_Data/nbackallsem01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_nbackallsem01 subset
tsfMRI_nbackallsem01 <- nbackallsem01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_nbackallsem01)
head(tsfMRI_nbackallsem01)

In [ ]:
# Save tsfMRI_nbackallsem01, remove intermediate files
saveRDS(tsfMRI_nbackallsem01, file="./CSI/Preprocessed/tsfMRI_nbackallsem01.rds")
rm(nbackallsem01)

### ABCD Task fMRI nBack Run 1 Beta Weights: _nbackr101_

In [ ]:
# Import partial nbackr101 data
nbackr101 <- read_delim("./ABCD_Data/nbackr101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_nbackr101 subset
tsfMRI_nbackr101 <- nbackr101 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_nbackr101)
head(tsfMRI_nbackr101)

In [ ]:
# Save tsfMRI_nbackr101, remove intermediate files
saveRDS(tsfMRI_nbackr101, file="./CSI/Preprocessed/tsfMRI_nbackr101.rds")
rm(nbackr101)

### ABCD Task fMRI nBack Run 1 Beta Weights Destrieux Parcellations: _tfncr1bwdp_

In [ ]:
# Import partial tfncr1bwdp data
abcd_tfncr1bwdp101 <- read_delim("./ABCD_Data/abcd_tfncr1bwdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfncr1bwdp201 <- read_delim("./ABCD_Data/abcd_tfncr1bwdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfncr1bwdp <- full_join(abcd_tfncr1bwdp101, abcd_tfncr1bwdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfncr1bwdp subset
tsfMRI_tfncr1bwdp <- tfncr1bwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfncr1bwdp_"))) %>%
    mutate_at(., vars(starts_with("tfncr1bwdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfncr1bwdp_")), mean, na.rm=T)

dim(tsfMRI_tfncr1bwdp)
head(tsfMRI_tfncr1bwdp)

In [ ]:
# Save tsfMRI_tfncr1bwdp, remove intermediate files
saveRDS(tsfMRI_tfncr1bwdp, file="./CSI/Preprocessed/tsfMRI_tfncr1bwdp.rds")
rm(abcd_tfncr1bwdp101, abcd_tfncr1bwdp201, tfncr1bwdp)

### ABCD Task fMRI nBack Run 1 SEM Destrieux Parcellations: _tfnbr1semdp_

In [ ]:
# Import partial tfnbr1semdp data
abcd_tfnbr1semdp101 <- read_delim("./ABCD_Data/abcd_tfnbr1semdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfnbr1semdp201 <- read_delim("./ABCD_Data/abcd_tfnbr1semdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfnbr1semdp <- full_join(abcd_tfnbr1semdp101, abcd_tfnbr1semdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfnbr1semdp subset
tsfMRI_tfnbr1semdp <- tfnbr1semdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfnbr1semdp_"))) %>%
    mutate_at(., vars(starts_with("tfnbr1semdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfnbr1semdp_")), mean, na.rm=T)

dim(tsfMRI_tfnbr1semdp)
head(tsfMRI_tfnbr1semdp)

In [ ]:
# Save tsfMRI_tfnbr1semdp, remove intermediate files
saveRDS(tsfMRI_tfnbr1semdp, file="./CSI/Preprocessed/tsfMRI_tfnbr1semdp.rds")
rm(abcd_tfnbr1semdp101, abcd_tfnbr1semdp201, tfnbr1semdp)

### ABCD Task fMRI nBack Run 2 Beta Weights Destrieux Parcellations: _tfnbr2bwdp_

In [ ]:
# Import partial tfnbr2bwdp data
abcd_tfnbr2bwdp101 <- read_delim("./ABCD_Data/abcd_tfnbr2bwdp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfnbr2bwdp201 <- read_delim("./ABCD_Data/abcd_tfnbr2bwdp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfnbr2bwdp <- full_join(abcd_tfnbr2bwdp101, abcd_tfnbr2bwdp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfnbr2bwdp subset
tsfMRI_tfnbr2bwdp <- tfnbr2bwdp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfnbr2bwdp_"))) %>%
    mutate_at(., vars(starts_with("tfnbr2bwdp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfnbr2bwdp_")), mean, na.rm=T)

dim(tsfMRI_tfnbr2bwdp)
head(tsfMRI_tfnbr2bwdp)

In [ ]:
# Save tsfMRI_tfnbr2bwdp, remove intermediate files
saveRDS(tsfMRI_tfnbr2bwdp, file="./CSI/Preprocessed/tsfMRI_tfnbr2bwdp.rds")
rm(abcd_tfnbr2bwdp101, abcd_tfnbr2bwdp201, tfnbr2bwdp)

### ABCD Task fMRI nBack Run 2 SEM Destrieux Parcellations: _tfnbr2dp_

In [ ]:
# Import partial tfnbr2dp data
abcd_tfnbr2dp101 <- read_delim("./ABCD_Data/abcd_tfnbr2dp101.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

abcd_tfnbr2dp201 <- read_delim("./ABCD_Data/abcd_tfnbr2dp201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

tfnbr2dp <- full_join(abcd_tfnbr2dp101, abcd_tfnbr2dp201, by = c("subjectkey" = "subjectkey"))

In [ ]:
# Create a tsfMRI_tfnbr2dp subset
tsfMRI_tfnbr2dp <- tfnbr2dp %>%
    filter(eventname.x %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfnbr2dp_"))) %>%
    mutate_at(., vars(starts_with("tfnbr2dp_")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfnbr2dp_")), mean, na.rm=T)

dim(tsfMRI_tfnbr2dp)
head(tsfMRI_tfnbr2dp)

In [ ]:
# Save tsfMRI_tfnbr2dp, remove intermediate files
saveRDS(tsfMRI_tfnbr2dp, file="./CSI/Preprocessed/tsfMRI_tfnbr2dp.rds")
rm(abcd_tfnbr2dp101, abcd_tfnbr2dp201, tfnbr2dp)

### ABCD Task fMRI nBack Run 1 Standard Error of the Mean: _nbackr1sem01_

In [ ]:
# Import partial nbackr1sem01 data
nbackr1sem01 <- read_delim("./ABCD_Data/nbackr1sem01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_nbackr1sem01 subset
tsfMRI_nbackr1sem01 <- nbackr1sem01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_nbackr1sem01)
head(tsfMRI_nbackr1sem01)

In [ ]:
# Save tsfMRI_nbackr1sem01, remove intermediate files
saveRDS(tsfMRI_nbackr1sem01, file="./CSI/Preprocessed/tsfMRI_nbackr1sem01.rds")
rm(nbackr1sem01)

### ABCD Task fMRI nBack Run 2 Beta Weights: _nbackr201_

In [ ]:
# Import partial nbackr201 data
nbackr201 <- read_delim("./ABCD_Data/nbackr201.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_nbackr201 subset
tsfMRI_nbackr201 <- nbackr201 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_nbackr201)
head(tsfMRI_nbackr201)

In [ ]:
# Save tsfMRI_nbackr201, remove intermediate files
saveRDS(tsfMRI_nbackr201, file="./CSI/Preprocessed/tsfMRI_nbackr201.rds")
rm(nbackr201)

### ABCD Task fMRI nBack Run 2 Standard Error of the Mean: _nbackr2sem01_

In [ ]:
# Import partial nbackr2sem01 data
nbackr2sem01 <- read_delim("./ABCD_Data/nbackr2sem01.txt", 
                           delim = "\t", 
                           escape_double = FALSE, 
                           col_types = "c", 
                           trim_ws = TRUE, 
                           na = c("", "NA"))[-1,]

In [ ]:
# Create a tsfMRI_nbackr2sem01 subset
tsfMRI_nbackr2sem01 <- nbackr2sem01 %>%
    filter(eventname %in% c("baseline_year_1_arm_1", "1_year_follow_up_y_arm_1", "2_year_follow_up_y_arm_1")) %>%
    select(c(subjectkey, starts_with("tfmri"))) %>%
    select(-ends_with("_visitid")) %>%
    mutate_at(., vars(starts_with("tfmri")), as.numeric) %>%
    group_by(subjectkey) %>%
    summarise_at(., vars(starts_with("tfmri")), mean, na.rm=T)

dim(tsfMRI_nbackr2sem01)
head(tsfMRI_nbackr2sem01)

In [ ]:
# Save tsfMRI_nbackr2sem01, remove intermediate files
saveRDS(tsfMRI_nbackr2sem01, file="./CSI/Preprocessed/tsfMRI_nbackr2sem01.rds")
rm(nbackr2sem01)

### Remove High Missingness

In [ ]:
# Count Infinities, NAs, and NaNs per variable
NAs_tsfMRI_mriqcrp <- apply(tsfMRI_mriqcrp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_mriqcrp <- apply(tsfMRI_mriqcrp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_postqc <- apply(tsfMRI_postqc, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_postqc <- apply(tsfMRI_postqc, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midabwdp <- apply(tsfMRI_midabwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midabwdp <- apply(tsfMRI_midabwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_fmriqc <- apply(fmriqc, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_fmriqc <- apply(fmriqc, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midaparcp <- apply(tsfMRI_midaparcp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midaparcp <- apply(tsfMRI_midaparcp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midasemdp <- apply(tsfMRI_midasemdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midasemdp <- apply(tsfMRI_midasemdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midasemp <- apply(tsfMRI_midasemp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midasemp <- apply(tsfMRI_midasemp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midr1bwdp <- apply(tsfMRI_midr1bwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midr1bwdp <- apply(tsfMRI_midr1bwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midr1bwp <- apply(tsfMRI_midr1bwp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midr1bwp <- apply(tsfMRI_midr1bwp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tmidr1semdp <- apply(tsfMRI_tmidr1semdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tmidr1semdp <- apply(tsfMRI_tmidr1semdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midsemp <- apply(tsfMRI_midsemp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midsemp <- apply(tsfMRI_midsemp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tr2bwdp <- apply(tsfMRI_tr2bwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tr2bwdp <- apply(tsfMRI_tr2bwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midr2bwp <- apply(tsfMRI_midr2bwp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midr2bwp <- apply(tsfMRI_midr2bwp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tr2semdp <- apply(tsfMRI_tr2semdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tr2semdp <- apply(tsfMRI_tr2semdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_midr2semp <- apply(tsfMRI_midr2semp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_midr2semp <- apply(tsfMRI_midr2semp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_mrisst02 <- apply(tsfMRI_mrisst02, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_mrisst02 <- apply(tsfMRI_mrisst02, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfsstabwdp <- apply(tsfMRI_tfsstabwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfsstabwdp <- apply(tsfMRI_tfsstabwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfsstasemdp <- apply(tsfMRI_tfsstasemdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfsstasemdp <- apply(tsfMRI_tfsstasemdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_mrisstsem01 <- apply(tsfMRI_mrisstsem01, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_mrisstsem01 <- apply(tsfMRI_mrisstsem01, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_mrisstr1bw01 <- apply(tsfMRI_mrisstr1bw01, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_mrisstr1bw01 <- apply(tsfMRI_mrisstr1bw01, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfsstr1bwdp <- apply(tsfMRI_tfsstr1bwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfsstr1bwdp <- apply(tsfMRI_tfsstr1bwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfsstr1semdp <- apply(tsfMRI_tfsstr1semdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfsstr1semdp <- apply(tsfMRI_tfsstr1semdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_mrisstr1sem01 <- apply(tsfMRI_mrisstr1sem01, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_mrisstr1sem01 <- apply(tsfMRI_mrisstr1sem01, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_mrisstr2bw01 <- apply(tsfMRI_mrisstr2bw01, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_mrisstr2bw01 <- apply(tsfMRI_mrisstr2bw01, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfsstr2bwdp <- apply(tsfMRI_tfsstr2bwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfsstr2bwdp <- apply(tsfMRI_tfsstr2bwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfsstr2semdp <- apply(tsfMRI_tfsstr2semdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfsstr2semdp <- apply(tsfMRI_tfsstr2semdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_mrisstr2bwsem01 <- apply(tsfMRI_mrisstr2bwsem01, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_mrisstr2bwsem01 <- apply(tsfMRI_mrisstr2bwsem01, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_nback_bwroi02 <- apply(tsfMRI_nback_bwroi02, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_nback_bwroi02 <- apply(tsfMRI_nback_bwroi02, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfabwdp <- apply(tsfMRI_tfabwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfabwdp <- apply(tsfMRI_tfabwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tnbasemdp <- apply(tsfMRI_tnbasemdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tnbasemdp <- apply(tsfMRI_tnbasemdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_nbackallsem01 <- apply(tsfMRI_nbackallsem01, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_nbackallsem01 <- apply(tsfMRI_nbackallsem01, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_nbackr101 <- apply(tsfMRI_nbackr101, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_nbackr101 <- apply(tsfMRI_nbackr101, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfncr1bwdp <- apply(tsfMRI_tfncr1bwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfncr1bwdp <- apply(tsfMRI_tfncr1bwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfnbr1semdp <- apply(tsfMRI_tfnbr1semdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfnbr1semdp <- apply(tsfMRI_tfnbr1semdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfnbr2bwdp <- apply(tsfMRI_tfnbr2bwdp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfnbr2bwdp <- apply(tsfMRI_tfnbr2bwdp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_tfnbr2dp <- apply(tsfMRI_tfnbr2dp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_tfnbr2dp <- apply(tsfMRI_tfnbr2dp, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_nbackr1sem01 <- apply(tsfMRI_nbackr1sem01, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_nbackr1sem01 <- apply(tsfMRI_nbackr1sem01, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_nbackr201 <- apply(tsfMRI_nbackr201, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_nbackr201 <- apply(tsfMRI_nbackr201, 2, function(x) {is.infinite(x)/length(x)})

NAs_tsfMRI_nbackr2sem01 <- apply(tsfMRI_nbackr2sem01, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
INFs_tsfMRI_nbackr2sem01 <- apply(tsfMRI_nbackr2sem01, 2, function(x) {is.infinite(x)/length(x)})

In [ ]:
# Max missingness
cat("NAs_tsfMRI_mriqcrp: ", round(max(NAs_tsfMRI_mriqcrp), 3),
   "\nINFs_tsfMRI_mriqcrp: ", round(max(INFs_tsfMRI_mriqcrp), 3),
    
   "\n\nNAs_tsfMRI_postqc: ", round(max(NAs_tsfMRI_postqc), 3),
   "\nINFs_tsfMRI_postqc: ", round(max(INFs_tsfMRI_postqc), 3),
    
   "\n\nNAs_tsfMRI_midabwdp: ", round(max(NAs_tsfMRI_midabwdp), 3),
   "\nINFs_tsfMRI_midabwdp: ", round(max(INFs_tsfMRI_midabwdp), 3),
    
   "\n\nNAs_fmriqc: ", round(max(NAs_fmriqc), 3),
   "\nINFs_fmriqc: ", round(max(INFs_fmriqc), 3),
    
   "\n\nNAs_tsfMRI_midaparcp: ", round(max(NAs_tsfMRI_midaparcp), 3),
   "\nINFs_tsfMRI_midaparcp: ", round(max(INFs_tsfMRI_midaparcp), 3),
    
   "\n\nNAs_tsfMRI_midasemdp: ", round(max(NAs_tsfMRI_midasemdp), 3),
   "\nINFs_tsfMRI_midasemdp: ", round(max(INFs_tsfMRI_midasemdp), 3),
    
   "\n\nNAs_tsfMRI_midasemp: ", round(max(NAs_tsfMRI_midasemp), 3),
   "\nINFs_tsfMRI_midasemp: ", round(max(INFs_tsfMRI_midasemp), 3),
    
   "\n\nNAs_tsfMRI_midr1bwdp: ", round(max(NAs_tsfMRI_midr1bwdp), 3),
   "\nINFs_tsfMRI_midr1bwdp: ", round(max(INFs_tsfMRI_midr1bwdp), 3),
    
   "\n\nNAs_tsfMRI_midr1bwp: ", round(max(NAs_tsfMRI_midr1bwp), 3),
   "\nINFs_tsfMRI_midr1bwp: ", round(max(INFs_tsfMRI_midr1bwp), 3),
    
   "\n\nNAs_tsfMRI_tmidr1semdp: ", round(max(NAs_tsfMRI_tmidr1semdp), 3),
   "\nINFs_tsfMRI_tmidr1semdp: ", round(max(INFs_tsfMRI_tmidr1semdp), 3),
    
   "\n\nNAs_tsfMRI_midsemp: ", round(max(NAs_tsfMRI_midsemp), 3),
   "\nINFs_tsfMRI_midsemp: ", round(max(INFs_tsfMRI_midsemp), 3),
    
   "\n\nNAs_tsfMRI_tr2bwdp: ", round(max(NAs_tsfMRI_tr2bwdp), 3),
   "\nINFs_tsfMRI_tr2bwdp: ", round(max(INFs_tsfMRI_tr2bwdp), 3),
    
   "\n\nNAs_tsfMRI_midr2bwp: ", round(max(NAs_tsfMRI_midr2bwp), 3),
   "\nINFs_tsfMRI_midr2bwp: ", round(max(INFs_tsfMRI_midr2bwp), 3),
    
   "\n\nNAs_tsfMRI_tr2semdp: ", round(max(NAs_tsfMRI_tr2semdp), 3),
   "\nINFs_tsfMRI_tr2semdp: ", round(max(INFs_tsfMRI_tr2semdp), 3),
    
   "\n\nNAs_tsfMRI_midr2semp: ", round(max(NAs_tsfMRI_midr2semp), 3),
   "\nINFs_tsfMRI_midr2semp: ", round(max(INFs_tsfMRI_midr2semp), 3),
    
   "\n\nNAs_tsfMRI_mrisst02: ", round(max(NAs_tsfMRI_mrisst02), 3),
   "\nINFs_tsfMRI_mrisst02: ", round(max(INFs_tsfMRI_mrisst02), 3),
    
   "\n\nNAs_tsfMRI_tfsstabwdp: ", round(max(NAs_tsfMRI_tfsstabwdp), 3),
   "\nINFs_tsfMRI_tfsstabwdp: ", round(max(INFs_tsfMRI_tfsstabwdp), 3),
    
   "\n\nNAs_tsfMRI_tfsstasemdp: ", round(max(NAs_tsfMRI_tfsstasemdp), 3),
   "\nINFs_tsfMRI_tfsstasemdp: ", round(max(INFs_tsfMRI_tfsstasemdp), 3),
    
   "\n\nNAs_tsfMRI_mrisstsem01: ", round(max(NAs_tsfMRI_mrisstsem01), 3),
   "\nINFs_tsfMRI_mrisstsem01: ", round(max(INFs_tsfMRI_mrisstsem01), 3),
    
   "\n\nNAs_tsfMRI_tfsstr1bwdp: ", round(max(NAs_tsfMRI_tfsstr1bwdp), 3),
   "\nINFs_tsfMRI_tfsstr1bwdp: ", round(max(INFs_tsfMRI_tfsstr1bwdp), 3),
    
   "\n\nNAs_tsfMRI_tfsstr1semdp: ", round(max(NAs_tsfMRI_tfsstr1semdp), 3),
   "\nINFs_tsfMRI_tfsstr1semdp: ", round(max(INFs_tsfMRI_tfsstr1semdp), 3),
    
   "\n\nNAs_tsfMRI_mrisstr1sem01: ", round(max(NAs_tsfMRI_mrisstr1sem01), 3),
   "\nINFs_tsfMRI_mrisstr1sem01: ", round(max(INFs_tsfMRI_mrisstr1sem01), 3),
    
   "\n\nNAs_tsfMRI_mrisstr2bw01: ", round(max(NAs_tsfMRI_mrisstr2bw01), 3),
   "\nINFs_tsfMRI_mrisstr2bw01: ", round(max(INFs_tsfMRI_mrisstr2bw01), 3),
    
   "\n\nNAs_tsfMRI_tfsstr2bwdp: ", round(max(NAs_tsfMRI_tfsstr2bwdp), 3),
   "\nINFs_tsfMRI_tfsstr2bwdp: ", round(max(INFs_tsfMRI_tfsstr2bwdp), 3),
    
   "\n\nNAs_tsfMRI_tfsstr2semdp: ", round(max(NAs_tsfMRI_tfsstr2semdp), 3),
   "\nINFs_tsfMRI_tfsstr2semdp: ", round(max(INFs_tsfMRI_tfsstr2semdp), 3),
    
   "\n\nNAs_tsfMRI_mrisstr2bwsem01: ", round(max(NAs_tsfMRI_mrisstr2bwsem01), 3),
   "\nINFs_tsfMRI_mrisstr2bwsem01: ", round(max(INFs_tsfMRI_mrisstr2bwsem01), 3),
    
   "\n\nNAs_tsfMRI_nback_bwroi02: ", round(max(NAs_tsfMRI_nback_bwroi02), 3),
   "\nINFs_tsfMRI_nback_bwroi02: ", round(max(INFs_tsfMRI_nback_bwroi02), 3),
    
   "\n\nNAs_tsfMRI_tfabwdp: ", round(max(NAs_tsfMRI_tfabwdp), 3),
   "\nINFs_tsfMRI_tfabwdp: ", round(max(INFs_tsfMRI_tfabwdp), 3),
    
   "\n\nNAs_tsfMRI_tnbasemdp: ", round(max(NAs_tsfMRI_tnbasemdp), 3),
   "\nINFs_tsfMRI_tnbasemdp: ", round(max(INFs_tsfMRI_tnbasemdp), 3),
    
   "\n\nNAs_tsfMRI_nbackallsem01: ", round(max(NAs_tsfMRI_nbackallsem01), 3),
   "\nINFs_tsfMRI_nbackallsem01: ", round(max(INFs_tsfMRI_nbackallsem01), 3),
    
   "\n\nNAs_tsfMRI_nbackr101: ", round(max(NAs_tsfMRI_nbackr101), 3),
   "\nINFs_tsfMRI_nbackr101: ", round(max(INFs_tsfMRI_nbackr101), 3),
    
   "\n\nNAs_tsfMRI_tfncr1bwdp: ", round(max(NAs_tsfMRI_tfncr1bwdp), 3),
   "\nINFs_tsfMRI_tfncr1bwdp: ", round(max(INFs_tsfMRI_tfncr1bwdp), 3),
    
   "\n\nNAs_tsfMRI_tfnbr1semdp: ", round(max(NAs_tsfMRI_tfnbr1semdp), 3),
   "\nINFs_tsfMRI_tfnbr1semdp: ", round(max(INFs_tsfMRI_tfnbr1semdp), 3),
    
   "\n\n:NAs_tsfMRI_tfnbr2bwdp ", round(max(NAs_tsfMRI_tfnbr2bwdp), 3),
   "\nINFs_tsfMRI_tfnbr2bwdp: ", round(max(INFs_tsfMRI_tfnbr2bwdp), 3),
    
   "\n\nNAs_tsfMRI_tfnbr2dp: ", round(max(NAs_tsfMRI_tfnbr2dp), 3),
   "\nINFs_tsfMRI_tfnbr2dp: ", round(max(INFs_tsfMRI_tfnbr2dp), 3),
    
   "\n\nNAs_tsfMRI_nbackr1sem01: ", round(max(NAs_tsfMRI_nbackr1sem01), 3),
   "\nINFs_tsfMRI_nbackr1sem01: ", round(max(INFs_tsfMRI_nbackr1sem01), 3),
    
   "\n\nNAs_tsfMRI_nbackr201: ", round(max(NAs_tsfMRI_nbackr201), 3),
   "\nINFs_tsfMRI_nbackr201: ", round(max(INFs_tsfMRI_nbackr201), 3),
    
   "\n\nNAs_tsfMRI_nbackr2sem01: ", round(max(NAs_tsfMRI_nbackr2sem01), 3),
   "\nINFs_tsfMRI_nbackr2sem01: ", round(max(INFs_tsfMRI_nbackr2sem01), 3))

In [ ]:
# Remove columns with high rates of NAs

tsfMRI_mriqcrp <- tsfMRI_mriqcrp[!colSums(is.na(tsfMRI_mriqcrp)) > 0.1]
tsfMRI_postqc <- tsfMRI_postqc[!colSums(is.na(tsfMRI_postqc)) > 0.1]
fmriqc <- fmriqc[!colSums(is.na(fmriqc)) > 0.1]

NAs_tsfMRI_mriqcrp <- apply(tsfMRI_mriqcrp, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
NAs_tsfMRI_postqc <- apply(tsfMRI_postqc, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})
NAs_fmriqc <- apply(fmriqc, 2, function(x) {sum(is.na(x)|is.nan(x))/length(x)})

cat("NAs_rsfMRI_mriqcrp: ", round(max(NAs_tsfMRI_mriqcrp), 3),    
   "\nNAs_rsfMRI_postqc: ", round(max(NAs_tsfMRI_postqc), 3),    
   "\nNAs_fmriqc: ", round(max(NAs_fmriqc), 3))

### Combine and Process Datasets

In [ ]:
# Calculate number of predictors for all datasets

cat(" 1. tsfMRI_mriqcrp:", dim(tsfMRI_mriqcrp)[2]-1,
    "\n 2. tsfMRI_postqc:", dim(tsfMRI_postqc)[2]-1,
    "\n 3. tsfMRI_midabwdp:", dim(tsfMRI_midabwdp)[2]-1,
    "\n 4. fmriqc:", dim(fmriqc)[2]-1,
    "\n 5. tsfMRI_midaparcp:", dim(tsfMRI_midaparcp)[2]-1,
    "\n 6. tsfMRI_midasemdp:", dim(tsfMRI_midasemdp)[2]-1,
    "\n 7. tsfMRI_midasemp:", dim(tsfMRI_midasemp)[2]-1,
    "\n 8. tsfMRI_midr1bwdp:", dim(tsfMRI_midr1bwdp)[2]-1,
    "\n 9. tsfMRI_midr1bwp:", dim(tsfMRI_midr1bwp)[2]-1,
    "\n10. tsfMRI_tmidr1semdp:", dim(tsfMRI_tmidr1semdp)[2]-1,
    "\n11. tsfMRI_midsemp", dim(tsfMRI_midsemp)[2]-1,
    "\n12. tsfMRI_tr2bwdp", dim(tsfMRI_tr2bwdp)[2]-1,
    "\n13. tsfMRI_midr2bwp", dim(tsfMRI_midr2bwp)[2]-1,
    "\n14. tsfMRI_tr2semdp", dim(tsfMRI_tr2semdp)[2]-1,
    "\n15. tsfMRI_midr2semp", dim(tsfMRI_midr2semp)[2]-1,
    "\n16. tsfMRI_mrisst02", dim(tsfMRI_mrisst02)[2]-1,
    "\n17. tsfMRI_tfsstabwdp", dim(tsfMRI_tfsstabwdp)[2]-1,
    "\n18. tsfMRI_tfsstasemdp", dim(tsfMRI_tfsstasemdp)[2]-1,
    "\n19. tsfMRI_mrisstsem01", dim(tsfMRI_mrisstsem01)[2]-1,
    "\n20. tsfMRI_mrisstr1bw01", dim(tsfMRI_mrisstr1bw01)[2]-1,
    "\n21. tsfMRI_tfsstr1bwdp", dim(tsfMRI_tfsstr1bwdp)[2]-1,
    "\n22. tsfMRI_tfsstr1semdp", dim(tsfMRI_tfsstr1semdp)[2]-1,
    "\n23. tsfMRI_mrisstr1sem01", dim(tsfMRI_mrisstr1sem01)[2]-1,
    "\n24. tsfMRI_mrisstr2bw01", dim(tsfMRI_mrisstr2bw01)[2]-1,
    "\n25. tsfMRI_tfsstr2bwdp", dim(tsfMRI_tfsstr2bwdp)[2]-1,
    "\n26. tsfMRI_tfsstr2semdp", dim(tsfMRI_tfsstr2semdp)[2]-1,
    "\n27. tsfMRI_mrisstr2bwsem01", dim(tsfMRI_mrisstr2bwsem01)[2]-1,
    "\n28. tsfMRI_nback_bwroi02", dim(tsfMRI_nback_bwroi02)[2]-1,
    "\n29. tsfMRI_tfabwdp", dim(tsfMRI_tfabwdp)[2]-1,
    "\n30. tsfMRI_tnbasemdp", dim(tsfMRI_tnbasemdp)[2]-1,
    "\n31. tsfMRI_nbackallsem01", dim(tsfMRI_nbackallsem01)[2]-1,
    "\n32. tsfMRI_nbackr101", dim(tsfMRI_nbackr101)[2]-1,
    "\n33. tsfMRI_tfncr1bwdp", dim(tsfMRI_tfncr1bwdp)[2]-1,
    "\n34. tsfMRI_tfnbr1semdp", dim(tsfMRI_tfnbr1semdp)[2]-1,
    "\n35. tsfMRI_tfnbr2bwdp", dim(tsfMRI_tfnbr2bwdp)[2]-1,
    "\n36. tsfMRI_tfnbr2dp", dim(tsfMRI_tfnbr2dp)[2]-1,
    "\n37. tsfMRI_nbackr1sem01", dim(tsfMRI_nbackr1sem01)[2]-1,
    "\n38. tsfMRI_nbackr201", dim(tsfMRI_nbackr201)[2]-1,
    "\n49. tsfMRI_nbackr2sem01", dim(tsfMRI_nbackr2sem01)[2]-1)

In [ ]:
# Combine non-qc and qc frames
sk <- purrr::reduce(list(tsfMRI_mriqcrp$subjectkey, tsfMRI_postqc$subjectkey, tsfMRI_midabwdp$subjectkey, fmriqc$subjectkey,
            tsfMRI_midaparcp$subjectkey, tsfMRI_midasemdp$subjectkey, tsfMRI_midasemp$subjectkey, tsfMRI_midr1bwdp$subjectkey,
            tsfMRI_midr1bwp$subjectkey, tsfMRI_tmidr1semdp$subjectkey, tsfMRI_midsemp$subjectkey, tsfMRI_tr2bwdp$subjectkey,
            tsfMRI_midr2bwp$subjectkey, tsfMRI_tr2semdp$subjectkey, tsfMRI_midr2semp$subjectkey, tsfMRI_mrisst02$subjectkey,
            tsfMRI_tfsstabwdp$subjectkey, tsfMRI_tfsstasemdp$subjectkey, tsfMRI_mrisstsem01$subjectkey, tsfMRI_mrisstr1bw01$subjectkey,
            tsfMRI_tfsstr1bwdp$subjectkey, tsfMRI_tfsstr1semdp$subjectkey, tsfMRI_mrisstr1sem01$subjectkey, tsfMRI_mrisstr2bw01$subjectkey,
            tsfMRI_tfsstr2bwdp$subjectkey, tsfMRI_tfsstr2semdp$subjectkey, tsfMRI_mrisstr2bwsem01$subjectkey, tsfMRI_nback_bwroi02$subjectkey,
            tsfMRI_tfabwdp$subjectkey, tsfMRI_tnbasemdp$subjectkey, tsfMRI_nbackallsem01$subjectkey, tsfMRI_nbackr101$subjectkey,
            tsfMRI_tfncr1bwdp$subjectkey, tsfMRI_tfnbr1semdp$subjectkey, tsfMRI_tfnbr2bwdp$subjectkey, tsfMRI_tfnbr2dp$subjectkey,
            tsfMRI_nbackr1sem01$subjectkey, tsfMRI_nbackr201$subjectkey, tsfMRI_nbackr2sem01$subjectkey), union)

tsfMRI <- data.frame(subjectkey = sk) %>%
    full_join(tsfMRI_mriqcrp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_postqc, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midabwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(fmriqc, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midaparcp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midasemdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midasemp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midr1bwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midr1bwp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tmidr1semdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midsemp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tr2bwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midr2bwp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tr2semdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_midr2semp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_mrisst02, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfsstabwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfsstasemdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_mrisstsem01, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_mrisstr1bw01, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfsstr1bwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfsstr1semdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_mrisstr1sem01, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_mrisstr2bw01, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfsstr2bwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfsstr2semdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_mrisstr2bwsem01, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_nback_bwroi02, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfabwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tnbasemdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_nbackallsem01, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_nbackr101, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfncr1bwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfnbr1semdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfnbr2bwdp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_tfnbr2dp, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_nbackr1sem01, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_nbackr201, by=c("subjectkey"="subjectkey")) %>%
    full_join(tsfMRI_nbackr2sem01, by=c("subjectkey"="subjectkey"))

dim(tsfMRI)
sum(is.na(tsfMRI))

In [ ]:
# Mean impute tsfMRI
for(i in 2:ncol(tsfMRI)){
  tsfMRI[is.na(tsfMRI[,i]), i] <- mean(tsfMRI[,i], na.rm = TRUE)
}

dim(tsfMRI)
sum(is.na(tsfMRI))

In [ ]:
# Remove variables with range of 0
Range <- apply(tsfMRI[,-1], 2, function(x) {range(x)[2]-range(x)[1]})
tsfMRI <- tsfMRI %>% select(-names(Range[Range==0]))

dim(tsfMRI)
head(tsfMRI)

In [ ]:
# Save fMRI data, remove all other variables
saveRDS(tsfMRI, file="./CSI/Preprocessed/kernel_tsfMRI.rds")
rm(list=setdiff(ls(), "tsfMRI"))
ls()

In [ ]:
# Split dataframe into 3 smaller dataframes
tsfMRI1 <- tsfMRI[ ,c(1, 2:9615)]

tsfMRI2 <- tsfMRI[ ,c(1, 9616:19231)]

tsfMRI3 <- tsfMRI[ ,c(1, 19232:28847)]

tsfMRI4 <- tsfMRI[ ,c(1, 28848:38462)]

In [ ]:
# Range-normalize data
tsfMRI1[,-1] <- apply(tsfMRI1[,-1], 2, function(x) {(x - min(x))/(max(x) - min(x))})
sum(is.na(tsfMRI1))

dim(tsfMRI1)
head(tsfMRI1)

In [ ]:
# Range-normalize data
tsfMRI2[,-1] <- apply(tsfMRI2[,-1], 2, function(x) {(x - min(x))/(max(x) - min(x))})
sum(is.na(tsfMRI2))

dim(tsfMRI2)
head(tsfMRI2)

In [ ]:
# Range-normalize data
tsfMRI3[,-1] <- apply(tsfMRI3[,-1], 2, function(x) {(x - min(x))/(max(x) - min(x))})
sum(is.na(tsfMRI3))

dim(tsfMRI3)
head(tsfMRI3)

In [ ]:
# Range-normalize data
tsfMRI4[,-1] <- apply(tsfMRI4[,-1], 2, function(x) {(x - min(x))/(max(x) - min(x))})
sum(is.na(tsfMRI4))

dim(tsfMRI4)
head(tsfMRI4)

In [ ]:
# Remove variables with low variances
dim(tsfMRI1)

Vars <- apply(tsfMRI1[,-1], 2, function(x) {var(x)})
tsfMRI1 <- tsfMRI1 %>% select(-names(Vars[Vars<0.001]))

dim(tsfMRI1)
head(tsfMRI1)

In [ ]:
# Remove variables with low variances
dim(tsfMRI2)

Vars <- apply(tsfMRI2[,-1], 2, function(x) {var(x)})
tsfMRI2 <- tsfMRI2 %>% select(-names(Vars[Vars<0.001]))

dim(tsfMRI2)
head(tsfMRI2)

In [ ]:
# Remove variables with low variances
dim(tsfMRI3)

Vars <- apply(tsfMRI3[,-1], 2, function(x) {var(x)})
tsfMRI3 <- tsfMRI3 %>% select(-names(Vars[Vars<0.001]))

dim(tsfMRI3)
head(tsfMRI3)

In [ ]:
# Remove variables with low variances
dim(tsfMRI4)

Vars <- apply(tsfMRI4[,-1], 2, function(x) {var(x)})
tsfMRI4 <- tsfMRI4 %>% select(-names(Vars[Vars<0.001]))

dim(tsfMRI4)
head(tsfMRI4)

In [ ]:
# Join modified frames
tsfMRI <- full_join(tsfMRI1, tsfMRI2, by = c("subjectkey" = "subjectkey")) %>%
    full_join(tsfMRI3, by = c("subjectkey" = "subjectkey")) %>%
    full_join(tsfMRI4, by = c("subjectkey" = "subjectkey"))

dim(tsfMRI)

In [ ]:
# Save fMRI data
saveRDS(tsfMRI, file="./CSI/Preprocessed/kernel_tsfMRI.rds")